In [53]:
import pandas as pd
import zipfile
import glob
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Load and preprocess the accelerometer data
def load_and_preprocess_data():
    zip_path = 'fall-dataset-all.zip' #zip_path = 'fall-dataset-all.zip'

    # Extract the CSV files from the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_files = [file for file in zip_ref.namelist() if file.endswith('.csv')]
        zip_ref.extractall(members=csv_files)

    # Read and concatenate the extracted CSV files into a DataFrame
    data = pd.concat([pd.read_csv(file, encoding='latin-1') for file in csv_files], ignore_index=True)
    accelerometer_data = data[["Acc(X)", "Acc(Y)", "Acc(Z)", "Rot(X)", "Rot(Y)", "Rot(Z)", "Pitch", "Roll", "Yaw", "Timestamp"]].values

    # Standardize the data
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(accelerometer_data)

    return standardized_data

# Define the augmentation function
def augment_function(sample):
    augmented_sample = apply_augmentation(sample)
    return augmented_sample

# Define augmentation functions
def apply_augmentation(sample):
    augmented_sample = sample.copy()

    # Noise Injection
    noise = np.random.normal(loc=0, scale=0.1, size=augmented_sample.shape)
    augmented_sample += noise

    # Time Shifting
    shift_amount = np.random.randint(low=1, high=len(augmented_sample))
    augmented_sample = np.roll(augmented_sample, shift_amount, axis=0)

    # Magnitude Scaling
    scaling_factor = np.random.uniform(low=0.8, high=1.2)
    augmented_sample *= scaling_factor

    return augmented_sample

# Define dataset class for accelerometer data
class AccelerometerDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        sample = self.data[index]
        augmented_sample_1 = augment_function(sample)
        augmented_sample_2 = augment_function(sample)
        return augmented_sample_1, augmented_sample_2

    def __len__(self):
        return len(self.data)

# Load and preprocess your accelerometer data
data = load_and_preprocess_data()

# Create the dataset
dataset = AccelerometerDataset(data)

# Create the data loader
batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the SimCLR model architecture
class SimCLRModel(nn.Module):
    def __init__(self, num_steps):
        super(SimCLRModel, self).__init__()
        self.embedding_size = 128
        self.num_steps = num_steps

        self.backbone = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )

        self.fc = nn.Linear(128 * (num_steps // 4), self.embedding_size)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        embedding = self.fc(x)
        return embedding

# Define the contrastive loss function
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=1.0):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, embeddings_1, embeddings_2):
        # Normalize the embeddings
        embeddings_1 = nn.functional.normalize(embeddings_1, dim=1)
        embeddings_2 = nn.functional.normalize(embeddings_2, dim=1)

        # Calculate cosine similarity between the embeddings
        similarities = torch.matmul(embeddings_1, embeddings_2.T) / self.temperature

        # Generate target labels (1 for positive pairs, 0 for negative pairs)
        labels = torch.arange(embeddings_1.size(0)).to(embeddings_1.device)

        # Calculate contrastive loss
        loss = nn.CrossEntropyLoss()(similarities, labels)

        return loss

# Initialize the SimCLR model, contrastive loss, and optimizer
num_steps = dataset[0][0].shape[0]
model = SimCLRModel(num_steps).double()
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    for batch in data_loader:
        # Clear the gradients
        optimizer.zero_grad()

        # Get the batch of augmented samples
        augmented_samples_1, augmented_samples_2 = batch

        # Reshape the input data to include the num_channels dimension
        augmented_samples_1 = augmented_samples_1.unsqueeze(1)
        augmented_samples_2 = augmented_samples_2.unsqueeze(1)

        # Forward pass
        embeddings_1 = model(augmented_samples_1.to(device).double())
        embeddings_2 = model(augmented_samples_2.to(device).double())

        # Calculate the contrastive loss
        loss = criterion(embeddings_1, embeddings_2)

        # Backward pass
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")


Epoch [1/10], Loss: 3.2126053250628477
Epoch [2/10], Loss: 3.2353333870859347
Epoch [3/10], Loss: 3.2186471160825527
Epoch [4/10], Loss: 3.2049791441694295
Epoch [5/10], Loss: 3.191840330442107
Epoch [6/10], Loss: 3.20446091197415
Epoch [7/10], Loss: 3.19332726100783
Epoch [8/10], Loss: 3.2021928981285703
Epoch [9/10], Loss: 3.195268111090501
Epoch [10/10], Loss: 3.1905640964652813


In [54]:
# Save the trained SimCLR model
torch.save(model.state_dict(), 'simclr_model.pth')

# Define the fall detection model
class FallDetectionModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FallDetectionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Create an instance of the fall detection model
num_classes = 2  # Specify the number of classes for fall detection
fall_model = FallDetectionModel(input_size=128, num_classes=num_classes)

# Define the fall detection loss function
fall_loss_function = nn.CrossEntropyLoss()

# Define the fall detection optimizer
fall_optimizer = optim.SGD(fall_model.parameters(), lr=0.001)

# Load the saved SimCLR model and extract learned embeddings
simclr_model = SimCLRModel(num_steps)
simclr_model.load_state_dict(torch.load('simclr_model.pth'))
simclr_model.eval()




SimCLRModel(
  (backbone): Sequential(
    (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=256, out_features=128, bias=True)
)

In [55]:
# To be incorporated into the previous load function
# Load and preprocess the accelerometer data
def load_and_preprocess(zip_path):

    # Extract the CSV files from the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_files = [file for file in zip_ref.namelist() if file.endswith('.csv')]
        zip_ref.extractall(members=csv_files)

    # Read and concatenate the extracted CSV files into a DataFrame
    data = pd.concat([pd.read_csv(file, encoding='latin-1') for file in csv_files], ignore_index=True)
    accelerometer_data = data[["Acc(X)", "Acc(Y)", "Acc(Z)", "Rot(X)", "Rot(Y)", "Rot(Z)", "Pitch", "Roll", "Yaw", "Timestamp"]].values
    labels = data["Fall"].values
    # Standardize the data
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(accelerometer_data)

    return standardized_data,labels

In [56]:

# # Load the fall-dataset-features csv files
# feature_files = glob.glob('fall-dataset-features/*.csv')
# features = []
# for feature_file in feature_files:
#     df = pd.read_csv(feature_file)
#     features.append(df.to_numpy())

# # Split the features into train and test sets
# train_features, test_features = np.split(features, [int(0.8 * len(features))])

# Load the fall-dataset-raw csv files
raw_files = "fall-dataset-raw.zip"#glob.glob('fall-dataset-raw/*.csv')
raw_data, labels = load_and_preprocess(raw_files)

# Split the raw data into train and test sets
train_raw_data, test_raw_data = np.split(raw_data, [int(0.8 * len(raw_data))])

# Split the raw data labels into train and test sets
train_labels, test_labels = np.split(labels, [int(0.8 * len(labels))])

# Define the train and test datasets
train_dataset = AccelerometerDataset(train_raw_data)
test_dataset = AccelerometerDataset(test_raw_data)


In [57]:
# Define the batch size
batch_size = 64

# Create the data loaders
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [58]:
simclr_model.to(device)

SimCLRModel(
  (backbone): Sequential(
    (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=256, out_features=128, bias=True)
)

In [59]:
train_embeddings = []
for batch in train_data_loader:
    augmented_samples_1, augmented_samples_2 = batch
    augmented_samples_1 = augmented_samples_1.unsqueeze(1).to(device)
    augmented_samples_2 = augmented_samples_2.unsqueeze(1).to(device)

    embeddings_1 = simclr_model(augmented_samples_1.to(device).float())
    embeddings_2 = simclr_model(augmented_samples_2.to(device).float())
    train_embeddings.append(embeddings_1)
    train_embeddings.append(embeddings_2)
train_embeddings = torch.cat(train_embeddings, dim=0)

test_embeddings = []
for batch in test_data_loader:
    augmented_samples_1, augmented_samples_2 = batch
    augmented_samples_1 = augmented_samples_1.unsqueeze(1).to(device)
    augmented_samples_2 = augmented_samples_2.unsqueeze(1).to(device)
    embeddings_1 = simclr_model(augmented_samples_1.to(device).float())
    embeddings_2 = simclr_model(augmented_samples_2.to(device).float())
    test_embeddings.append(embeddings_1)
    test_embeddings.append(embeddings_2)
test_embeddings = torch.cat(test_embeddings, dim=0)


In [60]:
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

In [ ]:
# Move the fall detection model to the same device as the embeddings
fall_model = fall_model.to(device)

# Training loop for the fall detection model
num_epochs = 10

for epoch in range(num_epochs):
    fall_model.train()
    total_loss = 0.0  # Track the total loss for the epoch

    for i in range(len(train_embeddings)):
        # Get the embeddings and labels for the current batch
        embeddings = train_embeddings[i].unsqueeze(0)
        labels = train_labels[i].unsqueeze(0)

        # Move embeddings and labels to the same device as the model
        embeddings = embeddings.to(device)
        labels = labels.to(device)

        # Convert the inputs and labels to torch.float32
        embeddings = embeddings.float()
        labels = labels.float()

        # Forward pass through the fall detection model
        outputs = fall_model(embeddings)

        # Calculate the fall detection loss
        fall_loss = fall_loss_function(outputs, labels.to(torch.int64))
        total_loss += fall_loss.item()

        # Zero the gradients
        fall_optimizer.zero_grad()

        # Backward pass and optimization
        fall_loss.backward(retain_graph=True)
        fall_optimizer.step()

    # Evaluate the fall detection model
    fall_model.eval()
    with torch.no_grad():
        # Perform evaluation on the testing set
        ...

    # Calculate the average loss for the epoch
    avg_loss = total_loss / len(train_embeddings)

    print(f"Epoch [{epoch+1}/{num_epochs}], Fall Detection Loss: {avg_loss}")


In [49]:
# # Move the fall detection model to the same device as the embeddings
# fall_model = fall_model.to(device)

# # Training loop for the fall detection model
# num_epochs = 10

# for epoch in range(num_epochs):
#     fall_model.train()
#     total_loss = 0.0  # Track the total loss for the epoch

#     for i in range(len(train_embeddings)):
#         # Get the embeddings and labels for the current batch
#         embeddings = train_embeddings[i].unsqueeze(0)
#         labels = train_labels[i].unsqueeze(0)

#         # Move embeddings and labels to the same device as the model
#         embeddings = embeddings.to(device)
#         labels = labels.to(device)

#         # Convert the inputs and labels to torch.float32
#         embeddings = embeddings.float()
#         labels = labels.float()

#         # Forward pass through the fall detection model
#         outputs = fall_model(embeddings)

#         # Calculate the fall detection loss
#         fall_loss = fall_loss_function(outputs, labels.to(torch.int64))
#         total_loss += fall_loss.item()

#         # Zero the gradients
#         fall_optimizer.zero_grad()

#         # Backward pass and optimization
#         fall_loss.backward(retain_graph=True)
#         fall_optimizer.step()

#     # Evaluate the fall detection model
#     fall_model.eval()
#     with torch.no_grad():
#         # Perform evaluation on the testing set
#         ...

#     # Calculate the average loss for the epoch
#     avg_loss = total_loss / len(train_embeddings)

#     print(f"Epoch [{epoch+1}/{num_epochs}], Fall Detection Loss: {avg_loss}")


KeyboardInterrupt: ignored

In [19]:
# # Create optimizer for the fall detection model
# fall_optimizer = optim.Adam(fall_model.parameters(), lr=0.001)


# # Move the fall optimizer  to the same device as the embeddings
# fall_optimizer = fall_optimizer.to(device)

# # Training loop for the fall detection model
# num_epochs = 10

# for epoch in range(num_epochs):
#     fall_model.train()
#     total_loss = 0.0  # Track the total loss for the epoch

#     for i in range(len(train_embeddings)):
#         # Get the embeddings and labels for the current batch
#         embeddings = train_embeddings[i].unsqueeze(0)
#         labels = train_labels[i].unsqueeze(0)

#         # Move embeddings and labels to the same device as the model
#         embeddings = embeddings.to(device)
#         labels = labels.to(device)

#         # Move the fall detection model to the same device as the embeddings
#         fall_model = fall_model.to(device)

#         # Convert the inputs and labels to torch.float32
#         embeddings = embeddings.float()
#         labels = labels.float()

#         # Forward pass through the fall detection model
#         outputs = fall_model(embeddings)

#         # Calculate the fall detection loss
#         fall_loss = fall_loss_function(outputs, labels.to(torch.int64))
#         total_loss += fall_loss.item()

#         # Zero the gradients
#         fall_optimizer.zero_grad()

#         # Backward pass and optimization
#         fall_loss.backward(retain_graph=True) #fall_loss.backward()
#         fall_optimizer.step()

#     # Evaluate the fall detection model
#     fall_model.eval()
#     with torch.no_grad():
#         # Perform evaluation on the testing set
#         ...

#     # Calculate the average loss for the epoch
#     avg_loss = total_loss / len(train_embeddings)

#     print(f"Epoch [{epoch+1}/{num_epochs}], Fall Detection Loss: {avg_loss}")


AttributeError: ignored

In [61]:
# # Training loop for the fall detection model
# num_epochs = 10

# for epoch in range(num_epochs):
#     fall_model.train()
#     for i in range(len(train_embeddings)):
#         # Clear the gradients
#         fall_optimizer.zero_grad()

#         # Get the embeddings and labels for the current batch
#         embeddings = train_embeddings[i].unsqueeze(0)
#         labels = train_labels[i].unsqueeze(0)

#         # Move embeddings and labels to the same device as the model
#         embeddings = embeddings.to(device)
#         labels = labels.to(device)

#         # Move the fall detection model to the same device as the embeddings
#         fall_model = fall_model.to(device)

#         # Forward pass through the fall detection model
#         outputs = fall_model(embeddings)

#         # Calculate the fall detection loss
#         fall_loss = fall_loss_function(outputs, labels)

#         # Backward pass and optimization
#         fall_loss.backward()
#         fall_optimizer.step()

#     # Evaluate the fall detection model
#     fall_model.eval()
#     with torch.no_grad():
#         # Perform evaluation on the testing set
#         ...

#     print(f"Epoch [{epoch+1}/{num_epochs}], Fall Detection Loss: {fall_loss.item()}")


RuntimeError: ignored

In [60]:
# # Training loop for the fall detection model
# num_epochs = 10

# for epoch in range(num_epochs):
#     fall_model.train()
#     for i in range(len(train_embeddings)):
#         # Clear the gradients
#         fall_optimizer.zero_grad()

#         # Get the embeddings and labels for the current batch
#         embeddings = train_embeddings[i].unsqueeze(0)
#         labels = train_labels[i].unsqueeze(0)

#         # Move embeddings and labels to the same device as the model
#         embeddings = embeddings.to(device)
#         labels = labels.to(device)

#         # Move the fall detection model to the same device as the embeddings
#         fall_model = fall_model.to(device)

#         # Forward pass through the fall detection model
#         outputs = fall_model(embeddings)

#         # Calculate the fall detection loss
#         fall_loss = fall_loss_function(outputs, labels)

#         # Backward pass and optimization
#         fall_loss.backward()
#         fall_optimizer.step()

#     # Evaluate the fall detection model
#     fall_model.eval()
#     with torch.no_grad():
#         # Perform evaluation on the testing set
#         ...

#     print(f"Epoch [{epoch+1}/{num_epochs}], Fall Detection Loss: {fall_loss.item()}")


RuntimeError: ignored

In [52]:
# train_embeddings = []
# with torch.no_grad():
#     for i in range(len(train_dataset)):
#         sample = train_dataset[i]
#         augmented_sample_1, augmented_sample_2 = sample
#         augmented_sample_1 = torch.from_numpy(augmented_sample_1).unsqueeze(0).to(device)
#         augmented_sample_2 = torch.from_numpy(augmented_sample_2).unsqueeze(0).to(device)
#         simclr_model.to(device)
#         embeddings_1 = simclr_model(augmented_sample_1.float())
#         embeddings_2 = simclr_model(augmented_sample_2.float())
#         train_embeddings.append(embeddings_1)
#         train_embeddings.append(embeddings_2)
# train_embeddings = torch.cat(train_embeddings, dim=0)

# test_embeddings = []
# with torch.no_grad():
#     for i in range(len(test_dataset)):
#         sample = test_dataset[i]
#         augmented_sample_1, augmented_sample_2 = sample
#         augmented_sample_1 = torch.from_numpy(augmented_sample_1).unsqueeze(0).to(device)
#         augmented_sample_2 = torch.from_numpy(augmented_sample_2).unsqueeze(0).to(device)
#         embeddings_1 = simclr_model(augmented_sample_1.float())
#         embeddings_2 = simclr_model(augmented_sample_2.float())
#         test_embeddings.append(embeddings_1)
#         test_embeddings.append(embeddings_2)
# test_embeddings = torch.cat(test_embeddings, dim=0)


In [53]:

# train_embeddings = []

# for batch in train_data_loader:
#     augmented_samples_1, augmented_samples_2 = batch
#     augmented_samples_1 = augmented_samples_1.unsqueeze(1)
#     augmented_samples_2 = augmented_samples_2.unsqueeze(1)
#     embeddings_1 = simclr_model(augmented_samples_1.to(device).float().to(device))
#     embeddings_2 = simclr_model(augmented_samples_2.to(device).float())
#     train_embeddings.append(embeddings_1)
#     train_embeddings.append(embeddings_2)
#     # train_labels.append(batch_labels)
# train_embeddings = torch.cat(train_embeddings, dim=0)
# # train_labels = torch.cat(train_labels, dim=0)

# test_embeddings = []

# for batch in test_data_loader:
#     augmented_samples_1, augmented_samples_2 = batch
#     augmented_samples_1 = augmented_samples_1.unsqueeze(1)
#     augmented_samples_2 = augmented_samples_2.unsqueeze(1)
#     embeddings_1 = simclr_model(augmented_samples_1.to(device).double())
#     embeddings_2 = simclr_model(augmented_samples_2.to(device).double())
#     test_embeddings.append(embeddings_1)
#     test_embeddings.append(embeddings_2)
#     # test_labels.append(batch_labels)
# test_embeddings = torch.cat(test_embeddings, dim=0)
# # test_labels = torch.cat(test_labels, dim=0)


In [ ]:
#########***********################

In [10]:
# # Save the trained SimCLR model
# torch.save(model.state_dict(), 'simclr_model.pth')

In [11]:
# import torch.nn.functional as F

# # Load the trained SimCLR model
# model = torch.load('simclr_model.pth')

# # Define the fall detection neural network
# class FallDetectionNet(nn.Module):
#     def __init__(self):
#         super(FallDetectionNet, self).__init__()
#         self.fc1 = nn.Linear(128, 64)
#         self.fc2 = nn.Linear(64, 1)

#     def forward(self, x):
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x



In [12]:
# # Initialize the fall detection neural network
# net = FallDetectionNet()


In [13]:

# # Define the loss function and optimizer
# criterion = nn.BCELoss()
# optimizer = optim.Adam(net.parameters(), lr=0.001)


OrderedDict([('backbone.0.weight',
              tensor([[[-2.3754e-01,  7.4393e-01,  2.2094e-01]],
              
                      [[ 5.2533e-02, -8.4310e-02,  4.2818e-01]],
              
                      [[-1.5580e-01, -1.6563e-01, -2.3675e-02]],
              
                      [[ 4.1152e-01, -8.3230e-02, -3.3861e-02]],
              
                      [[-5.0727e-01, -3.3134e-01,  6.5924e-01]],
              
                      [[ 2.8789e-01,  1.5657e-01, -2.7034e-01]],
              
                      [[ 6.5816e-02,  2.5802e-02,  3.4281e-02]],
              
                      [[ 9.9532e-01,  2.7249e-01,  1.9407e-01]],
              
                      [[-3.1774e-01,  1.7254e-02, -8.5120e-01]],
              
                      [[-1.5200e-01,  4.6153e-01, -1.8625e-01]],
              
                      [[ 4.3908e-01,  2.4581e-01, -4.2816e-02]],
              
                      [[ 1.8044e-01,  2.7117e-01, -4.2979e-03]],
              
     

In [20]:

# # Training loop
# num_epochs = 10

# for epoch in range(num_epochs):
#     for batch in data_loader:
#         # Clear the gradients
#         optimizer.zero_grad()

#         # Get the batch of augmented samples
#         augmented_samples_1, augmented_samples_2 = batch

#         # Reshape the input data to include the num_channels dimension
#         augmented_samples_1 = augmented_samples_1.unsqueeze(1)
#         augmented_samples_2 = augmented_samples_2.unsqueeze(1)

#         # Forward pass
#         embeddings_1 = model(augmented_samples_1.to(device).double())
#         embeddings_2 = model(augmented_samples_2.to(device).double())

#         # Calculate the fall probability
#         fall_probability = net(embeddings_1 - embeddings_2)

#         # Calculate the loss
#         loss = criterion(fall_probability, labels)

#         # Backward pass
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

TypeError: ignored

In [30]:
# # Convert the model to a SimCLR class object
# simclr_model = SimCLRModel()
# simclr_model.double()

# # Set the model parameters to the parameters of the trained model
# simclr_model.load_state_dict(torch.load('simclr_model.pth'))
# simclr_model.load_state_dict(torch.load('simclr_model.pth').double())

AttributeError: ignored

In [28]:

# # Training loop
# num_epochs = 10

# for epoch in range(num_epochs):
#     for batch in data_loader:
#         # Clear the gradients
#         optimizer.zero_grad()

#         # Get the batch of augmented samples
#         augmented_samples_1, augmented_samples_2 = batch

#         # Reshape the input data to include the num_channels dimension
#         augmented_samples_1 = augmented_samples_1.unsqueeze(1)
#         augmented_samples_2 = augmented_samples_2.unsqueeze(1)

#         # Forward pass
#         embeddings_1 = simclr_model(augmented_samples_1.to(device).double())
#         embeddings_2 = simclr_model(augmented_samples_2.to(device).double())

#         # Calculate the fall probability
#         fall_probability = net(embeddings_1 - embeddings_2)

#         # Calculate the loss
#         loss = criterion(fall_probability, labels)

#         # Backward pass
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

RuntimeError: ignored